# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 4

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task4`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [ ]:
!az login

## Step 1. Write the code

In [1]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

In [2]:
teamname="kodex_38"  # Update this field with your team name
task=["task4"]
slack_id="Jeevesh Krishna A"         # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [3]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task4_DumpMachineWrapper : qsharp.QSharpCallable = None
Task4_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 4 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [23]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;

// Task 4. 
// (input will contain 7 qubits)
operation original(input : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(input);
    for i in 0 .. 3 .. 2^N - 1 {
        ControlledOnInt(i, X)(input, target);
    }
}

operation mod3adder (reg : Qubit[]) : Unit is Adj+Ctl {
    let sum = reg[0];
    let carry = reg[1];
    ControlledOnInt(0, X)([carry], sum);
    ControlledOnInt(0, X)([sum], carry);
}

operation Task4(input : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    use output = Qubit[2] {
        within {
            for i in 0 .. Length(input) - 1 {
                if (i % 2 == 0) {
                    Controlled mod3adder([input[i]], output);
                } else {
                    Controlled Adjoint mod3adder([input[i]], output);
                }
            }
        } apply {
            ControlledOnInt(0, X)(output, target);
        }

    }
}

operation QubitArrayWrapperOperation (op : ((Qubit[], Qubit) => Unit is Adj), qs : Qubit[]) : Unit is Adj {        
    op(Most(qs), Tail(qs));
}

operation TestCompoundGate () : Unit {
    AssertOperationsEqualReferenced(7+1, QubitArrayWrapperOperation(Task4, _), QubitArrayWrapperOperation (original, _));
}

In [ ]:
TestCompoundGate();

In [24]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task4_DumpMachineWrapper() : Unit {
    let N = 7;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task4(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task4_ResourceEstimationWrapper() : Unit {
    let N = 7;
    use (input, target) = (Qubit[N], Qubit());
    Task4(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [25]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
qsharp.config["dump.truncateSmallAmplitudes"]=True
Task4_DumpMachineWrapper.simulate()

|00000000⟩	0.0883883476483185 + 0𝑖
|00000010⟩	0.0883883476483185 + 0𝑖
|00000100⟩	0.0883883476483185 + 0𝑖
|00000110⟩	0.0883883476483185 + 0𝑖
|00001000⟩	0.0883883476483185 + 0𝑖
|00001010⟩	0.0883883476483185 + 0𝑖
|00001100⟩	0.0883883476483185 + 0𝑖
|00001110⟩	0.0883883476483185 + 0𝑖
|00010000⟩	0.0883883476483185 + 0𝑖
|00010010⟩	0.0883883476483185 + 0𝑖
|00010100⟩	0.0883883476483185 + 0𝑖
|00010110⟩	0.0883883476483185 + 0𝑖
|00011000⟩	0.0883883476483185 + 0𝑖
|00011010⟩	0.0883883476483185 + 0𝑖
|00011100⟩	0.0883883476483185 + 0𝑖
|00011110⟩	0.0883883476483185 + 0𝑖
|00100000⟩	0.0883883476483185 + 0𝑖
|00100010⟩	0.0883883476483185 + 0𝑖
|00100100⟩	0.0883883476483185 + 0𝑖
|00100110⟩	0.0883883476483185 + 0𝑖
|00101000⟩	0.0883883476483185 + 0𝑖
|00101010⟩	0.0883883476483185 + 0𝑖
|00101100⟩	0.0883883476483185 + 0𝑖
|00101110⟩	0.0883883476483185 + 0𝑖
|00110000⟩	0.0883883476483185 + 0𝑖
|00110010⟩	0.0883883476483185 + 0𝑖
|00110100⟩	0.0883883476483185 + 0𝑖
|00110110⟩	0.0883883476483185 + 0𝑖
|00111000⟩	0.0883883476483185 + 0𝑖
|00111010⟩	0.0883883476483185 + 0𝑖
|00111100⟩	0.0883883476483185 + 0𝑖
|00111110⟩	0.0883883476483185 + 0𝑖
|01000000⟩	0.0883883476483185 + 0𝑖
|01000010⟩	0.0883883476483185 + 0𝑖
|01000100⟩	0.0883883476483185 + 0𝑖
|01000110⟩	0.0883883476483185 + 0𝑖
|01001000⟩	0.0883883476483185 + 0𝑖
|01001010⟩	0.0883883476483185 + 0𝑖
|01001100⟩	0.0883883476483185 + 0𝑖
|01001110⟩	0.0883883476483185 + 0𝑖
|01010000⟩	0.0883883476483185 + 0𝑖
|01010010⟩	0.0883883476483185 + 0𝑖
|01010100⟩	0.0883883476483185 + 0𝑖
|01010110⟩	0.0883883476483185 + 0𝑖
|01011000⟩	0.0883883476483185 + 0𝑖
|01011010⟩	0.0883883476483185 + 0𝑖
|01011100⟩	0.0883883476483185 + 0𝑖
|01011110⟩	0.0883883476483185 + 0𝑖
|01100000⟩	0.0883883476483185 + 0𝑖
|01100010⟩	0.0883883476483185 + 0𝑖
|01100100⟩	0.0883883476483185 + 0𝑖
|01100110⟩	0.0883883476483185 + 0𝑖
|01101000⟩	0.0883883476483185 + 0𝑖
|01101010⟩	0.0883883476483185 + 0𝑖
|01101100⟩	0.0883883476483185 + 0𝑖
|01101110⟩	0.0883883476483185 + 0𝑖
|01110000⟩	0.0883883476483185 + 0𝑖
|01110010⟩	0.0883883476483185 + 0𝑖
|01110100⟩	0.0883883476483185 + 0𝑖
|01110110⟩	0.0883883476483185 + 0𝑖
|01111000⟩	0.0883883476483185 + 0𝑖
|01111010⟩	0.0883883476483185 + 0𝑖
|01111100⟩	0.0883883476483185 + 0𝑖
|01111110⟩	0.0883883476483185 + 0𝑖
|10000000⟩	0.0883883476483185 + 0𝑖
|10000010⟩	0.0883883476483185 + 0𝑖
|10000100⟩	0.0883883476483185 + 0𝑖
|10000110⟩	0.0883883476483185 + 0𝑖
|10001000⟩	0.0883883476483185 + 0𝑖
|10001010⟩	0.0883883476483185 + 0𝑖
|10001100⟩	0.0883883476483185 + 0𝑖
|10001110⟩	0.0883883476483185 + 0𝑖
|10010000⟩	0.0883883476483185 + 0𝑖
|10010010⟩	0.0883883476483185 + 0𝑖
|10010100⟩	0.0883883476483185 + 0𝑖
|10010110⟩	0.0883883476483185 + 0𝑖
|10011000⟩	0.0883883476483185 + 0𝑖
|10011010⟩	0.0883883476483185 + 0𝑖
|10011100⟩	0.0883883476483185 + 0𝑖
|10011110⟩	0.0883883476483185 + 0𝑖
|10100000⟩	0.0883883476483185 + 0𝑖
|10100010⟩	0.0883883476483185 + 0𝑖
|10100100⟩	0.0883883476483185 + 0𝑖
|10100110⟩	0.0883883476483185 + 0𝑖
|10101000⟩	0.0883883476483185 + 0𝑖
|10101010⟩	0.0883883476483185 + 0𝑖
|10101100⟩	0.0883883476483185 + 0𝑖
|10101110⟩	0.0883883476483185 + 0𝑖
|10110000⟩	0.0883883476483185 + 0𝑖
|10110010⟩	0.0883883476483185 + 0𝑖
|10110100⟩	0.0883883476483185 + 0𝑖
|10110110⟩	0.0883883476483185 + 0𝑖
|10111000⟩	0.0883883476483185 + 0𝑖
|10111010⟩	0.0883883476483185 + 0𝑖
|10111100⟩	0.0883883476483185 + 0𝑖
|10111110⟩	0.0883883476483185 + 0𝑖
|11000000⟩	0.0883883476483185 + 0𝑖
|11000010⟩	0.0883883476483185 + 0𝑖
|11000100⟩	0.0883883476483185 + 0𝑖
|11000110⟩	0.0883883476483185 + 0𝑖
|11001000⟩	0.0883883476483185 + 0𝑖
|11001010⟩	0.0883883476483185 + 0𝑖
|11001100⟩	0.0883883476483185 + 0𝑖
|11001110⟩	0.0883883476483185 + 0𝑖
|11010000⟩	0.0883883476483185 + 0𝑖
|11010010⟩	0.0883883476483185 + 0𝑖
|11010100⟩	0.0883883476483185 + 0𝑖
|11010110⟩	0.0883883476483185 + 0𝑖
|11011000⟩	0.0883883476483185 + 0𝑖
|11011010⟩	0.0883883476483185 + 0𝑖
|11011100⟩	0.0883883476483185 + 0𝑖
|11011110⟩	0.0883883476483185 + 0𝑖
|11100000⟩	0.0883883476483185 + 0𝑖
|11100010⟩	0.0883883476483185 + 0𝑖
|11100100⟩

()

## Step 3. Evaluate the code using resource estimation

In [26]:
# If you're using this notebook in Azure Quantum hosted notebooks, remove the credential="CLI" parameter!
# If you're using this notebook in qBraid, keep it
qsharp.azure.connect(
    resourceId="/subscriptions/6d28879c-7c46-4b13-98e8-7da92da7e95e/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/iQuHackTest",
    location="eastus",
    credential="CLI")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace iQuHackTest in location eastus.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 265897},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 392346},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 3},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1', 'current_availability': {}, 'average_queue_time': 257411},
 {'id': 'quantinuum.hqs-lt-s1-apival', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s1-sim', 'current_availability': {}, 'average_queue_time': 4298},
 {'id': 'quantinuum.hqs-lt-s2-sim', 'current_availability': {}, 'average_queue_time': 138},
 {'id': 'quantinuum.hqs-lt', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current

In [27]:
qsharp.azure.target("microsoft.estimator")

Loading package Microsoft.Quantum.Providers.Core and dependencies...
Active target is now microsoft.estimator


{'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0}

In [28]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task4_ResourceEstimationWrapper, jobName="RE for the task 4")

Submitting Task4_ResourceEstimationWrapper to target microsoft.estimator...
Job successfully submitted.
   Job name: RE for the task 4
   Job ID: 5fb21790-6d97-4f4c-8d65-21230f8a3d83
Waiting up to 30 seconds for Azure Quantum job to complete...
[7:52:57 PM] Current job status: Executing
[7:53:02 PM] Current job status: Succeeded


In [29]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

Physical qubits,96876,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 6156 physical qubits to implement the algorithm logic, and 90720 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Runtime,82us 800ns,"Total runtime This is a runtime estimate (in nanosecond precision) for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (3us 600ns) multiplied by the 23 logical cycles to run the algorithm. If however the duration of a single T factory (here: 46us 800ns) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
Logical algorithmic qubits,38,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the \(Q_{\rm alg} = 13\) logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 38$ logical qubits."
Algorithmic depth,23,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 5 single-qubit measurements, the 0 arbitrary single-qubit rotations, and the 0 T gates, three multi-qubit measurements for each of the 1 CCZ and 5 CCiX gates in the input program, as well as No rotations in algorithm multi-qubit measurements for each of the 0 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,23,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 23. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Number of T states,24,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 0 T gates, four T states for each of the 1 CCZ and 5 CCiX gates, as well as No rotations in algorithm for each of the 0 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,14,Number of T factories capable of producing the demanded 24 T states during the algorithm's runtime The total number of T factories 14 that are executed in parallel is computed as \(\left\lceil\dfrac{24\;\text{T states} \cdot 46us 800ns\;\text{T factory duration}}{1\;\text{T states per T factory} \cdot 82us 800ns\;\text{algorithm runtime}}\right\rceil\)
Number of T factory invocations,2,"Number of times all T factories are invoked In order to prepare the 24 T states, the 14 copies of the T factory are repeatedly invoked 2 times."
Physical algorithmic qubits,6156,Number of physical qubits for the algorithm after layout The 6156 are the product of the 38 logical qubits after layout and the 162 physical qubits that encode a single logical qubit.
Physical T factory qubits,90720,"Number of physical qubits for the T factories Each T factory requires 6480 physical qubits and we run 14 in parallel, therefore we need $90720 = 6480 \cdot 14$ qubits."
Required logical qubit error rate,5.72e-7,The minimum logical qubit error rate required to run the algorithm within the error budget The minimum logical qubit error rate is obtained by dividing the logical error probability 5.00e-4 by the product of 38 logical qubits and the total cycle count 23.


In [30]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [31]:
evaluate_results(result)

Logical algorithmic qubits = 38
Algorithmic depth = 23
Score = 874


874